In [1]:
import warnings

warnings.filterwarnings("ignore")

import ast
import itertools
import json
import math
import os
import re
import sys
from collections import Counter
from copy import deepcopy
from functools import reduce
from pathlib import Path

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns
from networkx.algorithms.shortest_paths.weighted import dijkstra_path_length
from rich import print
from tqdm import tqdm

In [2]:
def plot_mtx(mtx, figsize=(10, 10)):
    fig, ax = plt.subplots(figsize=figsize)
    ax.matshow(mtx, cmap=plt.cm.Blues)
    plt.plot()

In [3]:
def read_file_to_str_li(fp, print_exp=True):
    with open(fp, "r") as f:
        lines = f.read().split("\n")
    if print_exp:
        print(f"Read from {fp}:")
        print(f"First line: {lines[0]} | Last line: {lines[-1]}")
        print("-" * 6)

    return lines


# define the function blocks
def convert_to_int(input_str):
    if input_str == "" or input_str == " ":
        return None
    return int(input_str)


def convert_to_str(input_str):
    return str(input_str)


# map the inputs to the function blocks
converts = {
    "i": convert_to_int,
    "s": convert_to_str,
}


def convert_str_li_to_other_li(
    str_li, pattern="i", per_letter=False, sep=" ", start_row=0, end_row=None
):
    """Convert a list of string to a list of other types

    pattern: a list of types for one item.
        'i' for int, 's' for string
        'si' means: convert the 1st item to string, the rest to integer
        If separated items are more than pattern items,
        use the last one from the parttern.
    if per_letter=True, ignore sep and separate item per letter
    """
    target_str_li = str_li[start_row:end_row]
    # find max item num
    max_item_num = 1
    if per_letter:
        max_item_num = max([len(s) for s in target_str_li])
    else:
        max_item_num = max([len(s.split(sep)) for s in target_str_li])

    # extend the pattern to the max itme num
    pattern = (
        pattern + f"{pattern[-1]}" * (max_item_num - len(pattern))
        if max_item_num > len(pattern)
        else pattern
    )

    # convert
    if per_letter:
        return [
            [converts[pattern[idx]](item) for idx, item in enumerate(s)]
            for s in target_str_li
        ]
    else:
        if sep == " ":
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split())]
                for s in target_str_li
            ]
        else:
            return [
                [converts[pattern[idx]](item) for idx, item in enumerate(s.split(sep))]
                for s in target_str_li
            ]

In [4]:
fp = "input.txt"
lines = read_file_to_str_li(fp)

print("Convert to:")

# head = convert_str_li_to_other_li(
#     lines, pattern="s", per_letter=True, sep=",", start_row=0, end_row=1
# )

# print(f"Head:\n{head}")
# print(f"First line: {head[0]}")
# print(f"Last line: {head[-1]}")

data = convert_str_li_to_other_li(
    lines, pattern="s", per_letter=False, sep=" ", start_row=None, end_row=None
)


print(f"First line: {data[0]}")
print(f"Last line: {data[-1]}")
print("-" * 6)

Read from input.txt:

First line: rtss: 5 | Last line: srqw: 3

------

Convert to:

First line: ['rtss:', '5']

Last line: ['srqw:', '3']

------

In [5]:
num_dict = {}
unknown_dict = {}
for line in data:
    if len(line) == 2:
        num_dict[line[0][:4]] = int(line[1])
    else:
        unknown_dict[line[0][:4]] = line[1:]

In [6]:
def get_root(num_dict, unknown_dict, check_direction="=="):
    while "root" in unknown_dict:
        pre_unknown_dict = unknown_dict.copy()
        for k, v in pre_unknown_dict.items():
            if (v[0] in num_dict) and (v[2] in num_dict):
                num_dict[k] = eval(
                    " ".join([str(num_dict[v[0]]), v[1], str(num_dict[v[2]])])
                )
                if k == "root":
                    if eval(f"{num_dict[v[0]]} {check_direction} {num_dict[v[2]]}"):
                        return True, int(num_dict[k])
                    else:
                        return False, int(num_dict[k])
                del unknown_dict[k]


def check_humn_num(num, num_dict, unknown_dict, check_direction):
    num_dict["humn"] = num
    return get_root(num_dict, unknown_dict, check_direction)[0]

In [7]:
%%time
q1_answer = get_root(num_dict.copy(), unknown_dict.copy())[1]
print(f"Answer to Q1: {q1_answer}")

Answer to Q1: 80326079210554

Wall time: 22 ms


In [8]:
%%time
left_value, right_value = 1, q1_answer
try_num = (left_value + right_value) // 2

while try_num != left_value:
    if check_humn_num(try_num, num_dict.copy(), unknown_dict.copy(), "<="):
        right_value = try_num
    else:
        left_value = try_num

    try_num = (left_value + right_value) // 2
for try_num in [left_value, right_value]:
    if check_humn_num(try_num, num_dict.copy(), unknown_dict.copy(), "=="):
        print(f"Answer to Q2: {try_num}")

Answer to Q2: 3617613952378

Wall time: 672 ms
